In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
basepath = "./data/Flowers/"
class1 = os.listdir(basepath + "Babi/")
class2 = os.listdir(basepath + "Calimerio/")

data = {'babi': class1[:10], 
        'calimerio': class2[:10], 
        'test': [class1[11], class2[11]]}

In [3]:
resnet50 = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

def _get_features(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
    img_data = tf.keras.utils.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = tf.keras.applications.resnet50.preprocess_input(img_data)
    resnet_features = resnet50.predict(img_data)
    return resnet_features

In [4]:
features = {"babi" : [], "calimerio" : [], "test" : []}
testimgs = []
for label, val in data.items():
    for k, each in enumerate(val):        
        if label == "test" and k == 0:
            img_path = basepath + "/Babi/" + each
            testimgs.append(img_path)
        elif label == "test" and k == 1:
            img_path = basepath + "/Calimerio/" + each
            testimgs.append(img_path)
        else: 
            img_path = basepath + label.title() + "/" + each
        feats = _get_features(img_path)
        features[label].append(feats.flatten())    

1/1 [==============================] - 0s 132ms/step


In [7]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

dataset = pd.DataFrame()
for label, feats in features.items():
    temp_df = pd.DataFrame(feats)
    temp_df['label'] = label
    dataset = pd.concat([dataset, pd.DataFrame()], ignore_index=True)
dataset.head()

y = dataset[dataset.label != 'test'].label
X = dataset[dataset.label != 'test'].drop('label', axis=1)

model = MLPClassifier(hidden_layer_sizes=(100, 10))
pipeline = Pipeline([('low_variance_filter', VarianceThreshold()), ('model', model)])
pipeline.fit(X, y)

preds = pipeline.predict(features['test'])

f, ax = plt.subplots(1, 2)
for i in range(2):
    ax[i].imshow(Image.open(testimgs[i]).resize((200, 200), Image.ANTIALIAS))
    ax[i].text(10, 180, 'Predicted: %s' % preds[i], color='k', backgroundcolor='red', alpha=0.8)
plt.show()

AttributeError: 'DataFrame' object has no attribute 'label'